## NetworkX

In [1]:
import numpy as np
import networkx as nx

with open("inputs/day17.txt", 'r') as fn:
    input = fn.read()


2D grid + dimension for prev dir, dimension for n steps
```
   y x p n
- (0,0,S,0) -> all directions with 1 step
            -> (0,1,R,1)
            -> (1,0,D,1)
- (0,1,R,1) -> directions U,D with 1 step, direction R with 2
            -> (0,2,R,2)
            -> (1,1,2,1)
- (0,2,R,2) -> directions U,D with 1 step, direction R with 3
            -> (0,3,R,3)
            -> (1,3,D,3)
- (0,3,R,3) -> directions U,D with 1 step
```

In [2]:
grid = np.genfromtxt("inputs/day17.txt", delimiter=1, dtype=int, comments=None)

In [3]:
def solve(grid, straight_range, turn_range):
    _G = nx.grid_2d_graph(*grid.shape).to_directed()
    G = nx.MultiDiGraph()

    directions = [(0,1), (0,-1), (1,0), (-1,0)]
    def change_dir(diff):
        opposite = (-diff[0], -diff[1])
        return [d for d in directions if d not in [diff, opposite]]

    for e in _G.edges():
        diff = (e[1][0]-e[0][0], e[1][1]-e[0][1])

        # Keep going, next lvl down
        for lvl in straight_range:    
            G.add_edge((e[0], diff, lvl), (e[1], diff, lvl+1), weight=grid[*e[1]])

        for lvl in turn_range:
            # Turn left or right, up to lvl 1
            for dir in change_dir(diff):
                G.add_edge((e[0], dir, lvl), (e[1], diff, 0), weight=grid[*e[1]])

    for dir in directions:
        G.add_edge('start', ((0,0), dir, 0), weight=0)
        for lvl in turn_range:
            G.add_edge(((grid.shape[0]-1, grid.shape[1]-1), dir, lvl), 'end', weight=0)

    print(nx.shortest_path_length(G, 'start', 'end', weight='weight'))

Part 1

In [4]:
solve(grid, range(3), range(3))

785


Part 2

In [5]:
solve(grid, range(10), range(3,10))

922
